![Astrofisica Computacional](../logo.png)

---
## 08. Introduction to `AstroPy`. Coordinates and Tables


Eduard Larrañaga (ealarranaga@unal.edu.co)

---

### About this notebook

In this notebook we present an introduction to the use coordinates and tables in `astropy`.

---

In [1]:
%matplotlib inline  
import matplotlib.pyplot as plt
import numpy as np
from astropy import units as u

## 1. Coordinates

The module [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) provides a framework to handle sky positions in various coordinate systems and transformations between them. The basic class to handle sky coordinates is [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html),

In [3]:
from astropy.coordinates import SkyCoord

We can define the position angle for longitude and latitude for a particular source, together with a keyword specifying a coordinate frame. For example, using the International Celestial Reference System 'icrs' and decimal degrees, we define the position of the crab nebula as

In [4]:
position_crab = SkyCoord(83.63 * u.deg,  22.01 * u.deg, frame='icrs')
position_crab

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63, 22.01)>

It is also possible to use `lists`, `arrays` or even `Quantities` to define the coordinates,

In [11]:
positions = SkyCoord([345., 234.3] * u.deg,  [-0.1, 0.2] * u.deg, frame='galactic')
positions

<SkyCoord (Galactic): (l, b) in deg
    [(345. , -0.1), (234.3,  0.2)]>

An interesting option is to define the angular position using strings with the `'hms'` and `'dms'` notation:

In [10]:
position_crab = SkyCoord('5h34m31.97s', '22d0m52.10s', frame='icrs')
position_crab

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63320833, 22.01447222)>

Alternatively, we can use the argument `unit`,

In [9]:
position_crab = SkyCoord('5:34:31.97', '22:0:52.10',
                         unit=(u.hour, u.deg), frame='icrs')
position_crab

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63320833, 22.01447222)>

### 1.1. Catalogues

A very convenient and easy way to get the coordinates of a particular source is by using the [Sesame](http://cds.u-strasbg.fr/cgi-bin/Sesame) database with the command `SkyCoord.from_name()`:

In [12]:
SkyCoord.from_name('Crab')

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63308333, 22.0145)>

To access the longitude and latitud angles individually we use the attributes `.lon`and `.lat`

In [17]:
position_crab.data.lon

<Longitude 5.57554722 hourangle>

In [18]:
position_crab.data.lat

<Latitude 22.01447222 deg>

### 1.2. Transformation between cordinate systems

In order to transform the coordinates from one coordinate system to another system we can use the command `SkyCoord.transform_to()`,

In [14]:
position_crab_galactic = position_crab.transform_to('galactic')
position_crab_galactic

<SkyCoord (Galactic): (l, b) in deg
    (184.55754381, -5.78427369)>

It is also possible to use the attributes `.galactic` or `.icrs` to perform the transformation:

In [15]:
position_crab.galactic

<SkyCoord (Galactic): (l, b) in deg
    (184.55754381, -5.78427369)>

In [16]:
position_crab.icrs

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63320833, 22.01447222)>

### 1.3. Measuring distances between positions in the sky

The angular distance between two [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) objects, can be found using the method [SkyCoord.separation()](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.separation).

For example, consider the source Sagittarius A* (Sgr A*), at the center of the Milky Way,

In [20]:
#position_saga = SkyCoord.from_name('Sag A*')
position_SgrA = SkyCoord(0 * u.deg, 0 * u.deg, frame='galactic')
position_SgrA

<SkyCoord (Galactic): (l, b) in deg
    (0., 0.)>

The distance from the Crab nebula to Sgr A* is

In [21]:
position_crab.separation(position_SgrA)

<Angle 172.64076197 deg>

In [48]:
position_crab

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63320833, 22.01447222)>

The inverse proble is also possible. In this case we want to compute a new position in the sky based on a given offset and position angle. For example, from the Crab nebula we can calculate

In [22]:
position_crab.directional_offset_by(
    separation=1 * u.deg, position_angle=0 * u.deg)

<SkyCoord (ICRS): (ra, dec) in deg
    (83.63320833, 23.01447222)>

### 1.4. ALT - AZ coordinates

When planning observations, it is convenient to transform the sky coordinates into a position in the horizontal coordinate system, given a location on earth and a time. We will use the functions [astropy.coordinates.Earthlocation](https://docs.astropy.org/en/stable/api/astropy.coordinates.EarthLocation.html) and [astropy.coordinates.AltAz](https://docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html),

In [23]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

The location of Bogotá, Colombia, is

In [25]:
Bogota = EarthLocation(lat=4.7110 * u.deg, lon=74.0721 * u.deg)
Bogota.geodetic

GeodeticLocation(lon=<Longitude 74.0721 deg>, lat=<Latitude 4.711 deg>, height=<Quantity -1.93903242e-09 m>)

and the local time is calculated using the [Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html) object:

In [27]:
now = Time.now()
print(now)

2021-09-22 13:33:54.876576


Now, we define a horizontal coordinate system using the [AltAz]([docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html) class and use it to convert from the sky coordinate,

In [29]:
altaz = AltAz(obstime=now, location=Bogota)
crab_altaz = position_crab.transform_to(altaz)
crab_altaz

<SkyCoord (AltAz: obstime=2021-09-22 13:33:54.876576, location=(1744462.32759558, 6112683.85272869, 520340.41574593) m, pressure=0.0 hPa, temperature=0.0 deg_C, relative_humidity=0.0, obswl=1.0 micron): (az, alt) in deg
    (28.56173846, -59.38685817)>

Note that we obtain the alt-az coordinates and aditional information about the local observation conditions.

## 2. Tables

Another interesting characteristic of `astropy` is the [Table](http://docs.astropy.org/en/stable/api/astropy.io.votable.tree.Table.html) class. This allows to handle data tables and data in .fits files.

In [2]:
from astropy.table import Table

Table objects are created by

In [3]:
table = Table()

We add columns to the table like we would add entries to a dictionary (Note the units for the coordinates!!)

In [4]:
table['Source_Name'] = ['Crab', 'Sag A*', 'Cas A', 'Vela Junior']
table['GalLon'] = [184.5575438, 0, 111.74169477, 266.25914205] * u.deg
table['GalLat'] = [-5.78427369, 0, -2.13544151, -1.21985818] * u.deg
table['Source_Class'] = ['pwn', 'unc', 'snr', 'snr']

By executing the following cell, we get a nicely formatted version of the table printed in the notebook:

In [5]:
table

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Crab,184.5575438,-5.78427369,pwn
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Vela Junior,266.25914205,-1.21985818,snr


### 2.1. Accessing rows and columns

The attribute `.colnames` gives the names of the columns,

In [6]:
table.colnames

['Source_Name', 'GalLon', 'GalLat', 'Source_Class']

To access individual columns we use their name,

In [7]:
table['GalLon']

184.5575438
0.0
111.74169477
266.25914205


In [8]:
table[['Source_Name', 'GalLat']]

Source_Name,GalLat
,deg
str11,float64
Crab,-5.78427369
Sag A*,0.0
Cas A,-2.13544151
Vela Junior,-1.21985818


It is also possible to get the column data as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity) using the `.quantity` property,

In [9]:
table['GalLon'].quantity

<Quantity [184.5575438 ,   0.        , 111.74169477, 266.25914205] deg>

Rows can be accessed using numpy indexing,

In [10]:
table[0:2]

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Crab,184.5575438,-5.78427369,pwn
Sag A*,0.0,0.0,unc


or by using a boolean numpy array for indexing,

In [11]:
selection = table['Source_Name'] == 'Crab'
table[selection]

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Crab,184.5575438,-5.78427369,pwn


More information about indexing can be consulted [here](http://docs.astropy.org/en/stable/table/indexing.html).

### 2.2. Indexing and Grouping

The method `.add_index()` allows to define an "index column" to access rows by the value contained in the index column. For example, we add the index corresponding to the "Source_Name" column,

In [12]:
table.add_index(colnames="Source_Name")

Now, it is possibleto access a particular row using the using the `.loc[]` syntax (as `pandas`dataframes):

In [13]:
table.loc["Cas A"]

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Cas A,111.74169477,-2.13544151,snr


In [14]:
table.loc[["Cas A", "Crab"]]

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Cas A,111.74169477,-2.13544151,snr
Crab,184.5575438,-5.78427369,pwn


It is also possible to group the rows by a given key column. The groups will be defined by the unique values contained in the column defined as key.

In [15]:
table_grouped = table.group_by("Source_Class")

for group in table_grouped.groups:
    print(group, "\n")

Source_Name    GalLon      GalLat   Source_Class
                deg         deg                 
----------- ----------- ----------- ------------
       Crab 184.5575438 -5.78427369          pwn 

Source_Name    GalLon       GalLat   Source_Class
                deg          deg                 
----------- ------------ ----------- ------------
      Cas A 111.74169477 -2.13544151          snr
Vela Junior 266.25914205 -1.21985818          snr 

Source_Name GalLon GalLat Source_Class
             deg    deg               
----------- ------ ------ ------------
     Sag A*    0.0    0.0          unc 



Each `group` created is again a `Table` object:

In [16]:
type(group)

astropy.table.table.Table

### 2.3. Reading / Writing tables to files
Astropy tables can be saved in many formats (for details see [here](http://docs.astropy.org/en/latest/io/unified.html#built-in-table-readers-writers)). 

In [18]:
table.write('example.fits', overwrite=True, format='fits')

In [19]:
Table.read('example.fits')

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
bytes11,float64,float64,bytes3
Crab,184.5575438,-5.78427369,pwn
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Vela Junior,266.25914205,-1.21985818,snr


### 2.4. Other operations

Other useful operations when working with Astropy tables.

- Sort by key:

In [47]:
table.sort('GalLon')

In [48]:
table

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Crab,184.5575438,-5.78427369,pwn
Vela Junior,266.25914205,-1.21985818,snr


Note that `.sort()` is an "in place operation" on the table, i.e. it changes the actual table.

- To remove a specific row by index:

In [6]:
table.remove_row(0)
table

Source_Name,GalLon,GalLat,Source_Class
,deg,deg,
str11,float64,float64,str3
Sag A*,0.0,0.0,unc
Cas A,111.74169477,-2.13544151,snr
Vela Junior,266.25914205,-1.21985818,snr


- Astropy tables also support row-wise iteration in Python loops:

In [7]:
for row in table:
    print(row['Source_Name'])

Sag A*
Cas A
Vela Junior


- Another useful feature for quickly inspecting the data contained in the table is the `.show_in_browser()` method:

In [8]:
table.show_in_browser(jsviewer=True)